**1. Load Essential Libraries & Setup**

In [ ]:
!nvidia-smi

Sun Jul  4 12:17:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### **2. Dataset**

**2.1. Download Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**2.2. Load Train Data**

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/sarcasm/reddit/balanced_politics_subset.csv')

In [ ]:
df.head()

,label,comment,parent_comment
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
1,0,I think a significant amount would be against ...,I bet if that money was poured into college de...
2,0,because it's what really bothers him... and it...,He actually acts like a moody emo girl on twit...
3,0,Conservatism as an ideology is for sure a reac...,"I still doubt that ""all conservatives stand fo..."
4,0,"Maybe not control, but certainly that is evide...",Today Russian media tweeted out that Wikileaks...


### Data Preprocess

In [ ]:
!pip install contractions

     |████████████████████████████████| 286kB 14.1MB/s 
     |████████████████████████████████| 327kB 28.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85409 sha256=a794dca7ba034f2a324b5b96171916f3bbfcbb207c97f04d404544fbfd1fae7b
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
import re
import contractions
def text_preprocessing(text):
    #fix contractions
    text = contractions.fix(text)

    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()


    return text

In [ ]:
df['comment']=df['comment'].apply(text_preprocessing)
df['parent_comment']=df['parent_comment'].apply(text_preprocessing)

In [ ]:
df['pair']=df['parent_comment']+str(" ")+df['comment']

In [ ]:
import numpy as np
df['pair'].replace('', np.nan, inplace=True)
df['pair'].replace('nan', np.nan, inplace=True)
df.dropna(subset=['pair'], inplace=True)

In [ ]:
df['pair']=df['pair'].astype(str)

In [ ]:
df=df.sample(frac=1)

In [ ]:
df.head()

,label,comment,parent_comment,pair
902,0,Wat,Why Are The Media Objectively Pro-Trump?,Why Are The Media Objectively Pro-Trump? Wat
18967,1,Posting facts is why Trump won,"You cannot just post mean facts like that man,...","You cannot just post mean facts like that man,..."
21507,1,"Sorry, we need to compromise.","how is this for a debate? Fuck you, you cannot...","how is this for a debate? Fuck you, you cannot..."
14652,0,they are just a subsidiary of the empire.,The Catholic league is not the Catholic empire.,The Catholic league is not the Catholic empire...
10407,0,"And not every rapist, terrorist, suicide bombe...","you are right, because every single US soldier...","you are right, because every single US soldier..."


In [ ]:
from sklearn.model_selection import train_test_split
X = df.pair.values
y = df.label.values

X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.50, random_state=42)

**3. Set up for training**

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


**4. Fine-tuning BERT**

**4.1 Install the Hugging Face Library**

In [ ]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 15.1MB/s 
     |████████████████████████████████| 5.6MB 19.6MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 1.2MB 44.1MB/s 
     |████████████████████████████████| 901kB 41.6MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 7.7MB 48.2MB/s 
ERROR: botocore 1.20.105 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


**4.2.1. BERT Tokenizer**

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

Before tokenizing, we need to specify the maximum length of our sentences.

In [ ]:
# Concatenate train data and test data
all_tweets = np.concatenate([X_train, X_test, X_val])

# Encode our concatenated data
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (693 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Max length:  951


Now let's tokenize our data.

In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 128

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  Why Are The Media Objectively Pro-Trump? Wat
Token IDs:  [101, 2339, 2024, 1996, 2865, 7863, 2135, 4013, 1011, 8398, 1029, 28194, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


**4.2.2. Create PyTorch DataLoader**

We will create an iterator for our dataset using the torch DataLoader class. This will help save on memory during training and boost the training speed.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

**4.3 Train Our Model**

**4.3.1. Create BertClassifier**

BERT-base consists of 12 transformer layers, each transformer layer takes in a list of token embeddings, and produces the same number of embeddings with the same hidden size (or dimensions) on the output. The output of the final transformer layer of the `[CLS]` token is used as the features of the sequence to feed a classifier.

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 128, 1

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out),
            nn.Sigmoid()
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 35 µs, sys: 1 µs, total: 36 µs
Wall time: 38.9 µs


**4.3.2. Optimizer & Learning Rate Scheduler**

To fine-tune our Bert Classifier, we need to create an optimizer using following hyper-parameters:

- Batch size: 16 or 32
- Learning rate (Adam): 5e-5, 3e-5 or 2e-5
- Number of epochs: 2, 3, 4

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

**4.3.3. Training Loop**

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.BCELoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch

            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")

    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        #preds = torch.argmax(logits, dim=1).flatten()

        preds=[1 if pred >=0.5 else 0 for pred in logits.flatten()]

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

Now, let's start training our BertClassifier!

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


ValueError: ignored

### **4.3.4. Evaluation on Validation Set**

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)

    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [ ]:
#uisng probability threshold as 0.50
predicted=np.argmax(probs,axis=1)

In [ ]:
print("---------------------------------------------")
print("Accuracy | Precision | Recall | F-1_score ")
print("---------------------------------------------")
print (" {:.3f}    | {:.3f}     | {:.3f}  | {:.3f}     ".format(accuracy_score(y_val,predicted), precision_score(y_val,predicted), recall_score(y_val,predicted), f1_score(y_val,predicted)))
print("---------------------------------------------")

---------------------------------------------
Accuracy | Precision | Recall | F-1_score 
---------------------------------------------
 0.771    | 0.758     | 0.793  | 0.775     
---------------------------------------------


**4.3.5. Train Our Model on the Entire Training Data**

In [ ]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
set_seed(42)
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, full_train_dataloader, epochs=2)

**4.4. Predictions on Test Set**

**4.4.1. Data Preparation**

Before making predictions on the test set, we need to redo processing and encoding steps done on the training data. Fortunately, we have written the `preprocessing_for_bert` function to do that for us.

In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data.comment)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

**4.4.2. Predictions**

Saracastic labels- 2702\
Non-Sarcastic labels- 1791\
60:40 Ratio


In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.80
preds = np.where(probs[:, 1] > threshold, 1, 0)

print("Number of comments predicted as sarcastic: ", preds.sum())

Taking threshold as 0.80 for a comment to be sarcastic

In [ ]:
f1_score(test['label'],preds)

**4.4.3. Data Preparation - Unbalanced & Balanced**

In [ ]:
test=df[35000:]
test_data = test[['comment']]
test.head()

In [ ]:
test_1 = test[test['label']==1]

In [ ]:
test_0 = test[test['label']==0]

In [ ]:
test_10=test_1[:199]
test_20=test_1[:440]
test_50=test_1[:1791]

In [ ]:
unbalanced_test_10=pd.concat([test_10,test_0]) # 10% and 90% ratio of sarcastic and non-sarcastic
unbalanced_test_20=pd.concat([test_20,test_0]) # 20% and 80% ratio of sarcastic and non-sarcastic
balanced_test_50=pd.concat([test_50,test_0]) # 50% and 50% ratio of sarcastic and non-sarcastic

In [ ]:
test_data_10 = unbalanced_test_10[['comment']]
test_data_20 = unbalanced_test_20[['comment']]
test_data_50 = balanced_test_50[['comment']]

**4.4.4. Predictions on Unbalanced Data-1**

Saracastic labels- 199\
Non-Sarcastic labels- 1791\
10:90 Ratio


In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data_10.comment)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.94
preds = np.where(probs[:, 1] > threshold, 1, 0)

print("Number of comments predicted as sarcastic: ", preds.sum())

In [ ]:
print("------------------------------------------")
print("Accuracy | Precision | Recall | F-1_score ")
print("------------------------------------------")
print (" {:.3f}    | {:.3f}     | {:.3f}  | {:.3f}     ".format(accuracy_score(unbalanced_test_10['label'],preds), precision_score(unbalanced_test_10['label'],preds), recall_score(unbalanced_test_10['label'],preds), f1_score(unbalanced_test_10['label'],preds)))
print("------------------------------------------")

**4.4.5. Predictions on Unbalanced Data-2**

Saracastic labels- 440\
Non-Sarcastic labels- 1791\
20:80 Ratio


In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data_20.comment)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.90
preds = np.where(probs[:, 1] > threshold, 1, 0)

print("Number of comments predicted as sarcastic: ", preds.sum())

In [ ]:
print("------------------------------------------")
print("Accuracy | Precision | Recall | F-1_score ")
print("------------------------------------------")
print (" {:.3f}    | {:.3f}     | {:.3f}  | {:.3f}     ".format(accuracy_score(unbalanced_test_20['label'],preds), precision_score(unbalanced_test_20['label'],preds), recall_score(unbalanced_test_20['label'],preds), f1_score(unbalanced_test_20['label'],preds)))
print("------------------------------------------")

**4.4.5. Predictions on Balanced Data-3**

Saracastic labels- 1791\
Non-Sarcastic labels- 1791\
50:50 Ratio


In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data_50.comment)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.71
preds = np.where(probs[:, 1] > threshold, 1, 0)

print("Number of comments predicted as sarcastic: ", preds.sum())

In [ ]:
print("------------------------------------------")
print("Accuracy | Precision | Recall | F-1_score ")
print("------------------------------------------")
print (" {:.3f}    | {:.3f}     | {:.3f}  | {:.3f}     ".format(accuracy_score(balanced_test_50['label'],preds), precision_score(balanced_test_50['label'],preds), recall_score(balanced_test_50['label'],preds), f1_score(balanced_test_50['label'],preds)))
print("------------------------------------------")